In [ ]:
import numpy as np
import os

import docopt
import trimesh
import glob

from trimesh.sample import sample_surface
from toolkit.src.renderer import render_cvcam
from toolkit.src.utility import show_img_arr

!export DISPLAY=172.17.0.1:0.0
os.environ['PYOPENGL_PLATFORM'] = 'egl'

In [ ]:
trimesh.__version__

In [ ]:
def main(input_path, output_dir='./out', scale=300, n=5000):
    """Passes command line arguments into utility function."""
#     arguments = docopt.docopt(__doc__)
#     input_path = arguments['<input_path>']
#     output_dir = arguments['--out']
#     scale = float(arguments['--scale'])
#     n = int(arguments['--n'])
    os.makedirs(output_dir, exist_ok=True)

    for path in glob.iglob(input_path):
        if not path.endswith('.obj'):
            print("Skipping: ", path)
            continue

        print("Processing file: ", path)
            
        mesh = trimesh.load(path)
        mesh.apply_scale(scale/mesh.scale)
        points = sample_surface(mesh, n)
        points -= points.mean(axis=0)
        output_path = os.path.join(output_dir, os.path.basename(path).split('.obj')[0] + '.npy')
        np.save(output_path, points)
    
    return output_path

In [ ]:
# input_path = './samples/*'
# out_path = main(input_path)

# Age Gender Labels

In [ ]:
# ADD PATH OF INFO FILE FROM FACESCAPE
ANNOT_FILE = '../../info_list_v1.txt'
f = open(ANNOT_FILE)
gender_map = {'f': 0, 'm': 1}

AGE_GENDER_DICT = dict()
for line in f:
    id_label, gender, age = line.split('\n')[0].split()
    try:
        AGE_GENDER_DICT[int(id_label)] = [ int(age), gender_map[gender] ]
    except:
        print("Skipping id: ", int(id_label), age, gender)
        continue
    
print(len(AGE_GENDER_DICT.keys()))

# Reading Training Data

In [ ]:
# ADD READ PATH FOR FACESCAPE
READ_ROOT_PATH = '../data/'
# ADD WRITE PATH FOR FACESCAPE
WRITE_ROOT_PATH = '../data_1024_out/'

ID_STARTS = [0, 101, 301, 501]
ID_ENDS = [100, 300, 500, 847]

# sampling hyperparameters
scale=300
# number of points to be sampled
n=1024 

for ID_START, ID_END in zip(ID_STARTS, ID_ENDS):
    idx_range = range(ID_START, ID_END+1)

    EXPR_NAME_DICT = dict()
    ALL_ANNOTS = dict() # .npy file: identity, age, gender, expression {id, name}

    print("START: ", ID_START, " END: ", ID_END)

    ANNOTS_FILE_SAVE_PATH = 'all_annots_{}_{}.npy'.format(ID_START, ID_END)
    print(ANNOTS_FILE_SAVE_PATH)
    
    total_written = 0

    for ix in idx_range:

        if ix not in AGE_GENDER_DICT:
            print("Skipping identity: ", ix)
            print("+++++++++++++++")

            continue

        READ_DIR_PATH = READ_ROOT_PATH + '{}/models_reg'.format(ix)
        WRITE_DIR_PATH = WRITE_ROOT_PATH + '{}/models_reg'.format(ix)

        os.makedirs(WRITE_DIR_PATH, exist_ok=True)

        if not os.path.exists(WRITE_DIR_PATH):
            # redundant check
            print("Missing output dir: ", WRITE_DIR_PATH)

        print("processing identity: ", ix, " from: ", READ_DIR_PATH)

        for name in os.listdir(READ_DIR_PATH):

            file_name = os.path.join(READ_DIR_PATH, name)

            if not file_name.endswith('.obj'):
                #print("Skipping: ", file_name)
                continue

            #print("Processing file: ", file_name)
            #print("------------------------------")

            mesh = trimesh.load(file_name)
            mesh.apply_scale(scale/mesh.scale)
            points = sample_surface(mesh, n)
            points -= points.mean(axis=0)

            output_name = os.path.basename(file_name).split('.obj')[0] + '.npy'

            output_path = os.path.join(WRITE_DIR_PATH, output_name)

            # Collecting annotations
            expr_id, expr_name = name.split('_')[0], '-'.join(name.split('_')[1:]).lower()

            if expr_id not in EXPR_NAME_DICT:
                EXPR_NAME_DICT[expr_id] = expr_name
            else:
                if expr_name != EXPR_NAME_DICT[expr_id]:
                    print("Debug! check mismatch.. ", expr_name, " : ", EXPR_NAME_DICT[expr_id])

            folder_id = ix
            age, gender = AGE_GENDER_DICT[folder_id]


            ALL_ANNOTS[output_path] = [folder_id, age, gender, expr_id, expr_name]

            np.save(output_path, points)

        num_written = len(os.listdir(WRITE_DIR_PATH))

        total_written += num_written

        print("Total files written: ", num_written, " in ", WRITE_DIR_PATH, " in all: ", total_written)
        print("----------------------------------")

    # Writing all
    print("Total files saved: ", len(ALL_ANNOTS.keys()), total_written, " at: ", ANNOTS_FILE_SAVE_PATH)
    np.save(ANNOTS_FILE_SAVE_PATH, ALL_ANNOTS)

## Combine Annotations

In [ ]:
import os
import numpy as np
ROOT = "./"
all_annotations = {}

for file in os.listdir(ROOT):
    if file.split(".")[-1] == "npy" and file.startswith("all_annots"):
        print(f"Processing {file}")
        x = np.load(os.path.join(ROOT, file), allow_pickle=True).item()
        
        for key in x.keys():
            all_annotations[key[3:]] = x[key]

# MODIFY SAVE PATH AS REQUIRED
np.save("./all_annotations1024.npy", all_annotations)

In [ ]:
test_dict = np.load("./all_annotations1024.npy", allow_pickle=True)

In [ ]:
len(test_dict.item().keys())

In [ ]:
max(test_dict.item().keys(), key=lambda x: int(x.split("/")[1]))

In [ ]:
min(test_dict.item().keys(), key=lambda x: int(x.split("/")[1]))

In [ ]:
# import os
# ROOT = "../"
# x = list(test_dict.item().keys())[0]
# y = np.load(os.path.join(ROOT, x), allow_pickle=True)

In [ ]:
# y.item()['points'].shape # bs, n, d